In [15]:
import sys
print("Python版本信息:")
print(sys.version)

import torch
print("torch版本信息:")
print(torch.__version__)

import sys
print("python路径:")
print(sys.executable)

Python版本信息:
3.10.6 (main, Oct 24 2022, 16:07:47) [GCC 11.2.0]
torch版本信息:
1.13.1+cu117
python路径:
/opt/anaconda3/envs/sd-webui/bin/python


In [16]:
from typing import Tuple, Optional
from denoising_diffusion.utils import gather

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

class DenoiseDiffusion:
    """
    ## Denoise Diffusion
    """

    def __init__(self, eps_model: nn.Module, n_steps: int, device: torch.device):
        """
        * `eps_model` is $\textcolor{lightgreen}{\epsilon_\theta}(x_t, t)$ model
        * `n_steps` is $t$
        * `device` is the device to place constants on
        """
        super().__init__()
        self.eps_model = eps_model

        # beta取值，线性等分向量
        self.beta = torch.linspace(0.0001, 0.02, n_steps).to(device)

        # alpha取值， 1-beta
        self.alpha = 1. - self.beta
        
        """
        b = torch.Tensor([1,2,3,4])
        a = torch.cumprod(b, dim=0)
        print(a) a=[1,2,6,24]
        """
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        
        #step默认值为1000
        self.n_steps = n_steps
        
        # sigma^2 = beta
        self.sigma2 = self.beta

In [ ]:
ddim = DenoiseDiffusion(None, n_steps=1000, device ="cuda",)
beta = nn.Parameter(ddim.beta.to(torch.float32), requires_grad=False)

print(beta)


In [ ]:
import numpy as np
n_steps= 1000
s_steps= 50
c = n_steps // s_steps
time_steps = np.asarray(list(range(0, n_steps, c))) + 1
print(time_steps)

In [ ]:

class DenoiseDiffusion:
    """
    ## Denoise Diffusion
    """

    def __init__(self, eps_model: nn.Module, n_steps: int, s_steps: int,device: torch.device):
        """
        * `eps_model` is $\textcolor{lightgreen}{\epsilon_\theta}(x_t, t)$ model
        * `n_steps` is $t$
        * `device` is the device to place constants on
        """
        super().__init__()
        self.eps_model = eps_model

        # beta取值，线性等分向量
        self.beta = torch.linspace(0.00085, 0.012, n_steps).to(device)

        # alpha取值， 1-beta
        self.alpha = 1. - self.beta
        
        """
        b = torch.Tensor([1,2,3,4])
        a = torch.cumprod(b, dim=0)
        print(a) a=[1,2,6,24]
        """
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        
        #step默认值为T=1000(n_steps), S=50(s_steps)
        self.n_steps = n_steps
        
        c = self.n_steps // s_steps
        self.time_steps = np.asarray(list(range(0, self.n_steps, c))) + 1

In [ ]:
ddim2 = DenoiseDiffusion(None, n_steps=1000, s_steps=50, device ="cuda",)

alpha_bar = ddim2.alpha_bar
ddim_alpha = alpha_bar[ddim2.time_steps].clone().to(torch.float32)

print(ddim2.time_steps)
print(ddim_alpha)

In [ ]:
ddim_alpha_prev = torch.cat([alpha_bar[0:1], alpha_bar[ddim2.time_steps[:-1]]])
ddim_alpha_prev

In [ ]:
ddim_eta=1
ddim_sigma = (ddim_eta *
                               ((1 - ddim_alpha_prev) / (1 - ddim_alpha) *
                                (1 - ddim_alpha / ddim_alpha_prev)) ** .5)

ddim_sigma

In [ ]:
ddim_alpha_prev = alpha_bar[ddim2.time_steps[:-1]]
print(ddim_alpha_prev)
ddim_eta=1
ddim_sigma = (ddim_eta *
                               ((1 - ddim_alpha_prev) / (1 - ddim_alpha) *
                                (1 - ddim_alpha / ddim_alpha_prev)) ** .5)

print(ddim_sigma)

In [9]:
from typing import Tuple, Optional
from denoising_diffusion.utils import gather

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import numpy as np

class DenoiseDiffusion3:
    """
    ## Denoise Diffusion
    """

    def __init__(self, eps_model: nn.Module, n_steps: int, s_steps: int, ddim_eta: float, device: torch.device):
        """
        * `eps_model` is $\textcolor{lightgreen}{\epsilon_\theta}(x_t, t)$ model
        * `n_steps` is $t$
        * `device` is the device to place constants on
        """
        super().__init__()
        self.eps_model = eps_model

        # beta取值，线性等分向量
        self.beta = torch.linspace(0.00085, 0.012, n_steps).to(device)

        # alpha取值， 1-beta
        self.alpha = 1. - self.beta
        
        """
        b = torch.Tensor([1,2,3,4])
        a = torch.cumprod(b, dim=0)
        print(a) a=[1,2,6,24]
        """
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        
        #step默认值为T=1000(n_steps), S=50(s_steps)
        self.n_steps = n_steps
        
        c = self.n_steps // s_steps
        self.time_steps = np.asarray(list(range(0, self.n_steps, c))) + 1
        
        
        # sigma^2 = beta in DDPM
        # self.sigma2 = self.beta

        # for DDIM
        self.ddim_alpha = self.alpha_bar[self.time_steps].clone().to(torch.float32)
        self.ddim_alpha_sqrt = torch.sqrt(self.ddim_alpha)
        self.ddim_alpha_prev = torch.cat([self.alpha_bar[0:1], self.alpha_bar[self.time_steps[:-1]]])
        self.ddim_sigma = (ddim_eta *
                               ((1 - self.ddim_alpha_prev) / (1 - self.ddim_alpha) *
                                (1 - self.ddim_alpha / self.ddim_alpha_prev)) ** .5)
        
        self.ddim_sqrt_one_minus_alpha = (1. - self.ddim_alpha) ** .5
        
ddim3 = DenoiseDiffusion3(None, n_steps=1000, s_steps=50, ddim_eta=1.0, device ="cuda", )
print(ddim3.ddim_alpha)
print(ddim3.ddim_alpha_prev)
print(ddim3.ddim_sigma)

tensor([0.9983, 0.9789, 0.9557, 0.9288, 0.8987, 0.8656, 0.8301, 0.7924, 0.7531,
        0.7126, 0.6712, 0.6294, 0.5875, 0.5460, 0.5052, 0.4653, 0.4266, 0.3894,
        0.3539, 0.3202, 0.2884, 0.2585, 0.2308, 0.2051, 0.1814, 0.1597, 0.1400,
        0.1222, 0.1062, 0.0918, 0.0791, 0.0678, 0.0578, 0.0491, 0.0416, 0.0350,
        0.0293, 0.0245, 0.0203, 0.0168, 0.0138, 0.0113, 0.0092, 0.0075, 0.0061,
        0.0049, 0.0039, 0.0031, 0.0025, 0.0020], device='cuda:0')
tensor([0.9991, 0.9983, 0.9789, 0.9557, 0.9288, 0.8987, 0.8656, 0.8301, 0.7924,
        0.7531, 0.7126, 0.6712, 0.6294, 0.5875, 0.5460, 0.5052, 0.4653, 0.4266,
        0.3894, 0.3539, 0.3202, 0.2884, 0.2585, 0.2308, 0.2051, 0.1814, 0.1597,
        0.1400, 0.1222, 0.1062, 0.0918, 0.0791, 0.0678, 0.0578, 0.0491, 0.0416,
        0.0350, 0.0293, 0.0245, 0.0203, 0.0168, 0.0138, 0.0113, 0.0092, 0.0075,
        0.0061, 0.0049, 0.0039, 0.0031, 0.0025], device='cuda:0')
tensor([0.0207, 0.0397, 0.1063, 0.1323, 0.1510, 0.1665, 0.1802, 0.19

In [14]:
skip_steps=10
time_steps = np.flip(ddim3.time_steps)[skip_steps:]
print(time_steps)

[781 761 741 721 701 681 661 641 621 601 581 561 541 521 501 481 461 441
 421 401 381 361 341 321 301 281 261 241 221 201 181 161 141 121 101  81
  61  41  21   1]


In [ ]:
torch.randint(0, len(self.time_steps), (11,), device=x0.device, dtype=torch.long)